In [18]:
# repo root
%cd /work

/work


In [20]:
!ls 

LICENSE    adapters  examples  paper	       scripts	tests
README.md  docker    out       pyproject.toml  src


In [24]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [32]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [33]:
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cards)


In [40]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [53]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [73]:
# Fig2 run
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --taus 0.2,0.4,0.6,0.8,0.9

[run_fig2] OK
  out: /work/paper/source_data/PANCAN_TP53_v1/out


In [74]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours

audit_log.tsv		    report.jsonl	       tau_0.40
claims.proposed.tsv	    report.md		       tau_0.60
distilled.tsv		    risk_coverage.tsv	       tau_0.80
distilled.with_modules.tsv  run_meta.json	       tau_0.90
evidence.normalized.tsv     sample_card.resolved.json  term_gene_edges.tsv
modules.tsv		    tau_0.20		       term_modules.tsv


In [75]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20
report   True 3334
audit    True 2799
modules  True 5506
edges    True 88967
terms    True 3288
distilled True 29940
risk     True 154
card     True 404
meta     True 2495

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (3, 35)
[report] decisions: {'PASS': 3}
[report] tau unique: [0.2]
[report] columns (top 25): ['schema_version', 'created_at', 'run_id', 'method', 'cancer', 'comparison', 'tau', 'claim_id', 'decision', 'survival', 'reason_codes', 'context_keys', 'disease', 'tissue', 'perturbation', 'gene_set_hash', 'benchmark_id', 'evidence_refs.module_ids', 'evidence_refs.term_ids', 'evidence_refs.gene_ids', 'evidence_refs.gene_set_hash', 'claim.entity_type', 'claim.entity_id', 'claim.entity_name', 'claim.direction']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comparison  tau       claim_id decision  survival claim.entity_

In [43]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
 [100%] Downloaded 33730584 bytes...
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
Warning messages:
1: In fgsea(pathways = pathways, stats = stats, nperm = nperm) :
  You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.
2: In preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam,  :
  There are ties in the preranked stats (0.02% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.
Error in setorderv(x, cols, order, na.last) : 
  some columns are not in

In [ ]:
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --taus 0.2,0.4,0.6,0.8,0.9